## $\color{Green}{\text{Check Individual Module (AR5K14) Results}}$

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import netCDF4 as nc
from pandas.plotting import table 
import xarray as xr

#### $\color{blue}{\text{List of Modules and Data to be used}}$

In [2]:
# Load FACTS workflow data as a dataframe.
df = pd.read_fwf('../001_GMD_IP_Modules_Data/ModuleTest_DataFile_List_ar5k14.txt',comment = '#')
df

,Component,Module,sub_Module,DataFile
0,GSAT,fair,temperature,temperature.fair.temperature_gsat.nc
1,GSAT,fair,temperature,temperature.fair.temperature_oceantemp.nc
2,GSAT,fair,temperature,temperature.fair.temperature_ohc.nc
3,TE,kopp14,sterodynamics,k14sterodynamics.kopp14.sterodynamics_globalsl.nc
4,TE,ipccar5,thermalexpansion,ar5TE.ipccar5.thermalexpansion_globalsl.nc
5,Glaciers,kopp14,glaciers,k14glaciers.kopp14.glaciers_globalsl.nc
6,Glaciers,ipccar5,glaciers,ar5glaciers.ipccar5.glaciers_globalsl.nc
7,GrIS,ipccar5,icesheets,ar5icesheets.ipccar5.icesheets_GIS_globalsl.nc
8,GrIS,kopp14,icesheets,k14icesheets.kopp14.icesheets_GIS_globalsl.nc
9,AIS,ipccar5,icesheets,ar5icesheets.ipccar5.icesheets_AIS_globalsl.nc


#### $\color{blue}{\text{Write nc data details to a file}}$

In [3]:
EXPDATE     = 221209
MODNO       = df.index.values;   # Number of datasets Module outputs
SCENARIO    = ['rcp26', 'rcp45', 'rcp85']; 
#        
file = open("ModuleData_Info_AR5K14.txt", "w")
#
for val in MODNO: 
    for scenario in SCENARIO:
        #
        MODULE = df["Module"][val];  SUB_MODULE = df["sub_Module"][val]; COMP = df["Component"][val];
        # Skip if data is absent.
        if df["DataFile"][val] == 'XXX':
           continue 
        #
        # Pick the data files & Import nc file to dataframe. 
        dataFOLDER  = '/projects/kopp/facts-experiments/{arg2}/ar5k14.{arg1}/output/'.format(arg1=scenario,arg2=EXPDATE)
        dataFILE    = 'ar5k14.{arg1}.'.format(arg1=scenario) + df["DataFile"][val]
        d_nc        = xr.open_dataset(dataFOLDER + dataFILE)
        d_nc
    file.write( "DataFILE ==> "+str(df["DataFile"][val])+"\n"+ str(d_nc.dims) + "\n" + str(d_nc.data_vars)+ "\n" + str(d_nc.coords)+"\n"+"-------------"+"\n"+"\n" )
        #
file.close()


#### $\color{Red}{\text{Load Module data from ==> facts-experiments/221209}}$

In [4]:
EXPDATE     = 221209
MODNO       = df.index.values;   # Number of datasets Module outputs
# MODNO       = np.arange(7);
SCENARIO    = ['rcp26', 'rcp45', 'rcp85']; 
YEAR        = 2100; # Percentile Year for the table.    
DATAEXCEPT = ['k14sterodynamics.kopp14.sterodynamics_globalsl.nc',
              'ar5TE.ipccar5.thermalexpansion_globalsl.nc',
              'k14glaciers.kopp14.glaciers_globalsl.nc',
              'deconto16.deconto16.AIS_AIS_globalsl.nc',
              'deconto16.deconto16.AIS_EAIS_globalsl.nc',
              'deconto16.deconto16.AIS_WAIS_globalsl.nc']
            
#
# Temp DF to store all .nc O/P.      
df_TEMP      = []
#        
for scenario in SCENARIO:
    # Define pd DF columns to append.
    PERCENTILE  = '%ile({arg2})'.format(arg2=scenario)
    percentile = np.nan
    df[PERCENTILE] = percentile
    df[PERCENTILE] = df[PERCENTILE].astype('object') 
    #
    for val in MODNO: 
        #
        MODULE = df["Module"][val];  SUB_MODULE = df["sub_Module"][val]; COMP = df["Component"][val]; DATAFILE = df["DataFile"][val];
        # Skip if data is absent.
        if df["DataFile"][val] == 'XXX':
           continue 
        #
        # Pick the data files & Import nc file to dataframe. 
        dataFOLDER  = '/projects/kopp/facts-experiments/{arg2}/ar5k14.{arg1}/output/'.format(arg1=scenario,arg2=EXPDATE)
        dataFILE    = 'ar5k14.{arg1}.'.format(arg1=scenario) + df["DataFile"][val]
        d_nc        = xr.open_dataset(dataFOLDER + dataFILE)
        d_nc
        #
        # Percentile calculation.
        percentList = [50, 17, 83]          #[50, 5, 17, 83, 95]
        # Find year index to pick SLC value
        if any(x in DATAFILE for x in DATAEXCEPT):
            Yind        = np.where(d_nc["year"].values == YEAR)[0][0]
        else: 
            Yind        = np.where(d_nc["years"].values == YEAR)[0][0]
        
        # 
        if SUB_MODULE == 'temperature':
            if  DATAFILE == 'temperature.fair.temperature_oceantemp.nc':
                    GMSL    = (d_nc["deep_ocean_temperature"][:,Yind,0].values)/1   
            elif DATAFILE == 'temperature.fair.temperature_ohc.nc':
                    GMSL    = (d_nc["ocean_heat_content"][:,Yind,0].values)/1   
            else:
                GMSL    = (d_nc["surface_temperature"][:,Yind,0].values)/1 
                #
        elif any(x in DATAFILE for x in DATAEXCEPT):
            GMSL    = (d_nc["samps"][Yind,:].values)/1

        else:
            GMSL    = (d_nc["sea_level_change"][:,Yind,0].values)/1       
        # Find Percentiles.
        pcntle = np.percentile(GMSL[:], percentList );    pcntle = np.around(pcntle,2)
        #
        # df.at[val,PERCENTILE] = [pcntle[0], pcntle[1],pcntle[2]]
        df.at[val,PERCENTILE] = f'{pcntle[0]:2.2f} ({pcntle[1]:2.2f} , {pcntle[2]:2.2f})' 
        # Print the OP.
print('GMSL Component Projections for ',d_nc["years"][Yind].values)    
#show(df)
df

GMSL Component Projections for  2100


,Component,Module,sub_Module,DataFile,%ile(rcp26),%ile(rcp45),%ile(rcp85)
0,GSAT,fair,temperature,temperature.fair.temperature_gsat.nc,"1.64 (1.38 , 1.94)","2.61 (2.24 , 3.02)","4.67 (4.02 , 5.33)"
1,GSAT,fair,temperature,temperature.fair.temperature_oceantemp.nc,"0.78 (0.53 , 1.12)","0.95 (0.63 , 1.35)","1.28 (0.85 , 1.87)"
2,GSAT,fair,temperature,temperature.fair.temperature_ohc.nc,1691964477708631397105664.00 (1443413383847857...,2131511045538785303461888.00 (1842339824910935...,3013558398439460085694464.00 (2664595593483396...
3,TE,kopp14,sterodynamics,k14sterodynamics.kopp14.sterodynamics_globalsl.nc,"161.50 (102.83 , 221.17)","227.50 (152.83 , 303.17)","337.50 (257.83 , 418.17)"
4,TE,ipccar5,thermalexpansion,ar5TE.ipccar5.thermalexpansion_globalsl.nc,"154.24 (127.00 , 176.92)","206.34 (176.52 , 231.18)","320.54 (278.39 , 355.64)"
5,Glaciers,kopp14,glaciers,k14glaciers.kopp14.glaciers_globalsl.nc,"112.07 (84.81 , 141.30)","130.21 (97.29 , 162.01)","171.07 (135.24 , 210.36)"
6,Glaciers,ipccar5,glaciers,ar5glaciers.ipccar5.glaciers_globalsl.nc,"90.50 (49.83 , 140.17)","114.00 (73.83 , 170.17)","157.00 (101.00 , 229.00)"
7,GrIS,ipccar5,icesheets,ar5icesheets.ipccar5.icesheets_GIS_globalsl.nc,"74.00 (53.00 , 97.17)","94.00 (63.83 , 119.17)","160.00 (112.49 , 204.00)"
8,GrIS,kopp14,icesheets,k14icesheets.kopp14.icesheets_GIS_globalsl.nc,"59.00 (34.00 , 108.17)","84.50 (34.83 , 145.34)","135.50 (72.83 , 245.51)"
9,AIS,ipccar5,icesheets,ar5icesheets.ipccar5.icesheets_AIS_globalsl.nc,"77.00 (-12.00 , 144.00)","67.00 (-19.17 , 135.34)","52.50 (-37.17 , 120.68)"


In [16]:
df.to_excel(r'Table_CheckModule_AR5K14_GMSL-Proj_2100.xlsx', index=False)
